<a href="https://colab.research.google.com/github/ndurumo254/GAME.PY/blob/master/MDN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Normal imports for everybody
from tensorflow import keras
import mdn
import numpy as np
from PIL import Image
import IPython.display

import os
import re
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import tensorflow as tf

from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.metrics import r2_score

In [ ]:
base_path = '/content/drive/MyDrive/Cement_Study'
csv_path = os.path.join(base_path, 'ccs.csv')
data_dir = base_path

In [ ]:
import pathlib
parent_dir = pathlib.Path(base_path)
image_count = len(list(data_dir.glob('*/*.bmp')))
print(image_count)

274


In [ ]:
#path configuration
parent_dir = '/content/drive/My Drive/Cement_Study'
#get category folder list
os.chdir(parent_dir)
category_list = list(filter(lambda x: os.path.isdir(x), os.listdir()))
for category in category_list:
  print(category)

test
A35
C45
A45


In [ ]:
#create training,validation,testing directories
data_set_dirs= ['train1','valid','test1']
for dsdirs in data_set_dirs:
  path = parent_dir + '/'+ dsdirs
  os.mkdir( path,755 )

In [ ]:
import json #create the json
import shutil #copy images to train, test and valid dirs
import os #files and dirs manipulation
import math #split calculate

In [ ]:
#define proportion of data
train_prop = 0.6
valid_prop = test_prop = (1-train_prop)/2
#function to split data of each category into trainning, validation and testing set
def create_dataset():
  for ii,cat in enumerate(category_list):    
    src_path = parent_dir + '/' + cat
    dest_dir1 = parent_dir+'/train/'+str(ii)
    dest_dir2 = parent_dir+'/valid/'+str(ii)
    dest_dir3 = parent_dir+'/test/'+str(ii)
    
    dest_dirs_list = [dest_dir1,dest_dir2,dest_dir3]
    for dirs in dest_dirs_list:
      os.mkdir(dirs,755 )
    
    #get files' names list from respective directories
    os.chdir(src_path)
    files = [f for f in os.listdir() if os.path.isfile(f)]
    
    #get training, testing and validation files count
    train_count = math.ceil(train_prop*len(files))
    valid_count = int((len(files)-train_count)/2)
    test_count = valid_count
    
    #get files to segragate for train,test and validation data set
    train_data_list = files[0: train_count]
    valid_data_list = files[train_count+1:train_count+1+valid_count]  
    test_data_list =  files[train_count+valid_count:]
       
  
    for train_data in train_data_list:
      train_path = src_path + '/' + train_data
      shutil.copy(train_path,dest_dir1)
    
    for valid_data in valid_data_list:
      valid_path = src_path + '/' + valid_data
      shutil.copy(valid_path,dest_dir2)
    
    for test_data in test_data_list:
      test_path = src_path + '/' + test_data
      shutil.copy(test_path,dest_dir3)

In [ ]:
#save category data as dictionary in a json file
cat_data = {}
for ix,cat in enumerate(category_list):
  cat_data[ix] = cat
with open('/content/drive/My Drive/Cement_Study/cat_data.json', 'w') as outfile:  
    json.dump(cat_data , outfile)

In [ ]:
pip install keras-mdn-layer

  Created wheel for keras-mdn-layer: filename=keras_mdn_layer-0.3.0-py3-none-any.whl size=7053 sha256=bee797bc3696f9f7ac3ad2f6117fdbe69c8845ddb622b58df4ab1dfcc410e1f3
  Stored in directory: /root/.cache/pip/wheels/40/9e/cf/ae1c857140eddaf1ece320918ca707c042ccf7cc441a9a389f
Successfully built keras-mdn-layer


**SPLIT** **DATA** 

In [ ]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test 

NameError: ignored

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, batch_input_shape=(None, 1), activation='relu'))
model.add(keras.layers.Dense(N_HIDDEN, activation='relu'))
model.add(mdn.MDN(1, N_MIXES))
model.compile(loss=mdn.get_mixture_loss_func(1, N_MIXES), optimizer=keras.optimizers.Adam())
model.summary()

history = model.fit(x=x_data, y=y_data, batch_size=128, epochs=500, validation_split=0.15)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 15)                30        
_________________________________________________________________
dense_4 (Dense)              (None, 15)                240       
_________________________________________________________________
mdn_2 (MDN)                  (None, 30)                480       
Total params: 750
Trainable params: 750
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500


KeyboardInterrupt: ignored

In [ ]:
x_test = np.float32(np.arange(-15, 15, 0.01))
NTEST = x_test.size
print("Testing:", NTEST, "samples.")
x_test = x_test.reshape(NTEST, 1)  # needs to be a matrix, not a vector

Testing: 3000 samples.


In [ ]:
# Make predictions from the model
y_test = model.predict(x_test)
# y_test contains parameters for distributions, not actual points on the graph.
# To find points on the graph, we need to sample from each distribution.

In [ ]:
# Sample from the predicted distributions
y_samples = np.apply_along_axis(mdn.sample_from_output, 1, y_test, 1, N_MIXES, temp=1.0)

In [ ]:
# Split up the mixture parameters (for future fun)
mus = np.apply_along_axis((lambda a: a[:N_MIXES]), 1, y_test)
sigs = np.apply_along_axis((lambda a: a[N_MIXES:2*N_MIXES]), 1, y_test)
pis = np.apply_along_axis((lambda a: mdn.softmax(a[2*N_MIXES:])), 1, y_test)

print("Done.")

Done.
